# MongoDB Aggregation Pipeline In Python

Learn about the various stages and configurations you can create to configure an Aggregation Pipeline

Stages: [match](#match), [project](#project), [unset](#unset), [limit](#limit), [skip](#skip), [sort](#sort), [count](#count), [sortByCount](#sortByCount), [unwind](#unwind), [group](#group), [addFields](#addFields), [sample](#sample), [lookup](#lookup), [unionWith](#unionWith), [out](#out), [merge](#merge)

Operators: [size](#size-(operator)), [in](#in-(operator)), [arrayElemAt](#arrayElemAt-(operator)), [first](#first-(operator)), [count](#count-(accumulator-operator)), [sum](#sum-(accumulator-operator)), [first, last](#first,-last-(accumulator-operators)), [push](#push-(accumulator-operator)), [addToSet](#addToSet-(accumulator-operator)), [regexMatch](#regexMatch-(operator)), [cond](#cond-(operator)), [Date](#Date-Operators), [expr](#expr-(operator)), [ifNull](#ifNull-(operator)), [type](#type-(operator)), [switch](#switch-(operator))

In [1]:
from pymongo import MongoClient

In [2]:
mongodb_uri = "mongodb://localhost:27017/"
db_name = "aggregation_test"
client = MongoClient(mongodb_uri)
db = client[db_name]

### Helper Function

In [3]:
def print_cursor(cursor):
    for document in cursor:
        print(document, end="\n\n")

### Inserting Some Sample Data

In [4]:
import insert_aggregation_sample_data as iasd
iasd.insert_data(mongodb_uri, db_name)

Entries already exist in the aggregation_test database in the users, products, or orders collection. Insert commands aborted.


### match

In [5]:
match = db.products.aggregate([
    {"$match": {"name": "Pens"}}
])

In [7]:
print_cursor(match)

{'_id': ObjectId('642d06472c53af7ae4ee5b89'), 'name': 'Pens', 'seller_id': ObjectId('642d06472c53af7ae4ee5b82'), 'tags': ['Office', 'School']}



In [10]:
match = db.products.aggregate([
    {"$match": {"$or": [{"tags": "Beauty"}, {"tags": "Home"}]}}
])

In [11]:
print_cursor(match)

{'_id': ObjectId('642d06472c53af7ae4ee5b87'), 'name': 'Mug', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('642d06472c53af7ae4ee5b88'), 'name': 'Moisturizer', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b8a'), 'name': 'Face Cleanser', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b8b'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('642d06472c53af7ae4ee5b84'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b8c'), 'name': 'Eyeliner', 'seller_id': ObjectId('642d06472c53af7ae4ee5b85'), 'tags': ['Beauty']}



### project

In [14]:
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "product_name": "$name", "tags": 1}}
])

In [15]:
print_cursor(project_cursor)

{'tags': ['Home', 'Kitchen'], 'product_name': 'Mug'}

{'tags': ['Beauty'], 'product_name': 'Moisturizer'}

{'tags': ['Office', 'School'], 'product_name': 'Pens'}

{'tags': ['Beauty'], 'product_name': 'Face Cleanser'}

{'tags': ['Beauty'], 'product_name': 'Concealer Makeup'}

{'tags': ['Beauty'], 'product_name': 'Eyeliner'}



In [16]:
match_project_cursor = db.products.aggregate([
    {"$match": {"name": "Pens"}},
    {"$project": {"_id": 0, "product_name": "$name", "tags": 1}}
])

In [17]:
print_cursor(match_project_cursor)

{'tags': ['Office', 'School'], 'product_name': 'Pens'}



### unset

In [18]:
unset_cursor = db.products.aggregate([
    {"$unset": ["_id", "seller_id"]}
])

In [19]:
print_cursor(unset_cursor)

{'name': 'Mug', 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'tags': ['Beauty']}

{'name': 'Pens', 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'tags': ['Beauty']}

{'name': 'Eyeliner', 'tags': ['Beauty']}



In [20]:
unset_cursor = db.products.aggregate([
    {"$match": {"name": "Pens"}},
    {"$unset": ["_id", "seller_id"]}
])

In [21]:
print_cursor(unset_cursor)

{'name': 'Pens', 'tags': ['Office', 'School']}



### limit

In [22]:
limit_cursor = db.products.aggregate([
    {"$limit": 3},
])

In [23]:
print_cursor(limit_cursor)

{'_id': ObjectId('642d06472c53af7ae4ee5b87'), 'name': 'Mug', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('642d06472c53af7ae4ee5b88'), 'name': 'Moisturizer', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b89'), 'name': 'Pens', 'seller_id': ObjectId('642d06472c53af7ae4ee5b82'), 'tags': ['Office', 'School']}



### skip

In [24]:
skip_cursor = db.products.aggregate([
    {"$skip": 2},
])

In [25]:
print_cursor(skip_cursor)

{'_id': ObjectId('642d06472c53af7ae4ee5b89'), 'name': 'Pens', 'seller_id': ObjectId('642d06472c53af7ae4ee5b82'), 'tags': ['Office', 'School']}

{'_id': ObjectId('642d06472c53af7ae4ee5b8a'), 'name': 'Face Cleanser', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b8b'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('642d06472c53af7ae4ee5b84'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b8c'), 'name': 'Eyeliner', 'seller_id': ObjectId('642d06472c53af7ae4ee5b85'), 'tags': ['Beauty']}



In [27]:
skip_limit_cursor = db.products.aggregate([
    {"$skip": 2},
    {"$limit": 3}
])

In [28]:
print_cursor(skip_limit_cursor)

{'_id': ObjectId('642d06472c53af7ae4ee5b89'), 'name': 'Pens', 'seller_id': ObjectId('642d06472c53af7ae4ee5b82'), 'tags': ['Office', 'School']}

{'_id': ObjectId('642d06472c53af7ae4ee5b8a'), 'name': 'Face Cleanser', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b8b'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('642d06472c53af7ae4ee5b84'), 'tags': ['Beauty']}



In [29]:
skip_limit_cursor = db.products.aggregate([
    {"$limit": 3},
    {"$skip": 2}
])
print_cursor(skip_limit_cursor)

{'_id': ObjectId('642d06472c53af7ae4ee5b89'), 'name': 'Pens', 'seller_id': ObjectId('642d06472c53af7ae4ee5b82'), 'tags': ['Office', 'School']}



### sort

In [30]:
sort_cursor = db.products.aggregate([
    {"$sort": {"name": 1}}
])

In [31]:
print_cursor(sort_cursor)

{'_id': ObjectId('642d06472c53af7ae4ee5b8b'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('642d06472c53af7ae4ee5b84'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b8c'), 'name': 'Eyeliner', 'seller_id': ObjectId('642d06472c53af7ae4ee5b85'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b8a'), 'name': 'Face Cleanser', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b88'), 'name': 'Moisturizer', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b87'), 'name': 'Mug', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('642d06472c53af7ae4ee5b89'), 'name': 'Pens', 'seller_id': ObjectId('642d06472c53af7ae4ee5b82'), 'tags': ['Office', 'School']}



### count

In [32]:
count_cursor = db.products.aggregate([
    {"$match": {"tags": "Beauty"}},
    {"$count": "beauty_products_count"}
])
print_cursor(count_cursor)

{'beauty_products_count': 4}



### sortByCount

In [33]:
count_cursor = db.products.aggregate([
    {"$sortByCount": "$tags"}
])
print_cursor(count_cursor)

{'_id': ['Beauty'], 'count': 4}

{'_id': ['Home', 'Kitchen'], 'count': 1}

{'_id': ['Office', 'School'], 'count': 1}



### size (operator)

In [34]:
size_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "num_tags": {"$size": "$tags"}, "tags": "$tags"}}
])
print_cursor(size_cursor)

{'name': 'Mug', 'num_tags': 2, 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'num_tags': 1, 'tags': ['Beauty']}

{'name': 'Pens', 'num_tags': 2, 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'num_tags': 1, 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'num_tags': 1, 'tags': ['Beauty']}

{'name': 'Eyeliner', 'num_tags': 1, 'tags': ['Beauty']}



### in (operator)

In [35]:
in_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "is_beauty_product": {"$in": ["Beauty", "$tags"]}, "tags": "$tags"}}
])
print_cursor(in_cursor)

{'name': 'Mug', 'is_beauty_product': False, 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'is_beauty_product': True, 'tags': ['Beauty']}

{'name': 'Pens', 'is_beauty_product': False, 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'is_beauty_product': True, 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'is_beauty_product': True, 'tags': ['Beauty']}

{'name': 'Eyeliner', 'is_beauty_product': True, 'tags': ['Beauty']}



### arrayElemAt (operator)

In [36]:
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "first_tag": {"$arrayElemAt": ["$tags", 0]}, "tags": "$tags"}}
])
print_cursor(project_cursor)

{'name': 'Mug', 'first_tag': 'Home', 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Pens', 'first_tag': 'Office', 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'first_tag': 'Beauty', 'tags': ['Beauty']}

{'name': 'Eyeliner', 'first_tag': 'Beauty', 'tags': ['Beauty']}



In [39]:
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "first_tag": {"$arrayElemAt": ["$tags", 1]}, "tags": "$tags"}}
])
print_cursor(project_cursor)

{'name': 'Mug', 'first_tag': 'Kitchen', 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'tags': ['Beauty']}

{'name': 'Pens', 'first_tag': 'School', 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'tags': ['Beauty']}

{'name': 'Eyeliner', 'tags': ['Beauty']}



### first (operator)

In [41]:
project_cursor = db.products.aggregate([
    {"$project": {"_id": 0, "name": 1, "first_tag": "$tags", "tags": "$tags"}}
])
print_cursor(project_cursor)

{'name': 'Mug', 'first_tag': ['Home', 'Kitchen'], 'tags': ['Home', 'Kitchen']}

{'name': 'Moisturizer', 'first_tag': ['Beauty'], 'tags': ['Beauty']}

{'name': 'Pens', 'first_tag': ['Office', 'School'], 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'first_tag': ['Beauty'], 'tags': ['Beauty']}

{'name': 'Concealer Makeup', 'first_tag': ['Beauty'], 'tags': ['Beauty']}

{'name': 'Eyeliner', 'first_tag': ['Beauty'], 'tags': ['Beauty']}



### unwind

In [42]:
unwind_cursor = db.products.aggregate([
    {"$unwind": "$tags"},
    {"$unset": ["_id", "seller_id"]}
])
print_cursor(unwind_cursor)

{'name': 'Mug', 'tags': 'Home'}

{'name': 'Mug', 'tags': 'Kitchen'}

{'name': 'Moisturizer', 'tags': 'Beauty'}

{'name': 'Pens', 'tags': 'Office'}

{'name': 'Pens', 'tags': 'School'}

{'name': 'Face Cleanser', 'tags': 'Beauty'}

{'name': 'Concealer Makeup', 'tags': 'Beauty'}

{'name': 'Eyeliner', 'tags': 'Beauty'}



In [45]:
unwind_cursor = db.orders.aggregate([
    {"$unwind": "$items"}
])
print_cursor(unwind_cursor)

{'_id': ObjectId('642d06472c53af7ae4ee5b8d'), 'items': {'product_id': ObjectId('642d06472c53af7ae4ee5b88'), 'quantity': 1}}

{'_id': ObjectId('642d06472c53af7ae4ee5b8d'), 'items': {'product_id': ObjectId('642d06472c53af7ae4ee5b8a'), 'quantity': 1}}

{'_id': ObjectId('642d06472c53af7ae4ee5b8e'), 'items': {'product_id': ObjectId('642d06472c53af7ae4ee5b8b'), 'quantity': 1}}

{'_id': ObjectId('642d06472c53af7ae4ee5b8e'), 'items': {'product_id': ObjectId('642d06472c53af7ae4ee5b8c'), 'quantity': 1}}

{'_id': ObjectId('642d06472c53af7ae4ee5b8f'), 'items': {'product_id': ObjectId('642d06472c53af7ae4ee5b89'), 'quantity': 5}}

{'_id': ObjectId('642d06472c53af7ae4ee5b8f'), 'items': {'product_id': ObjectId('642d06472c53af7ae4ee5b87'), 'quantity': 1}}

{'_id': ObjectId('642d06472c53af7ae4ee5b90'), 'items': {'product_id': ObjectId('642d06472c53af7ae4ee5b88'), 'quantity': 2}}

{'_id': ObjectId('642d06472c53af7ae4ee5b90'), 'items': {'product_id': ObjectId('642d06472c53af7ae4ee5b8c'), 'quantity': 1}}



In [47]:
unwind_cursor = db.products.aggregate([
    {"$match": {"tags": {"$size": 2}}},
    {"$unwind": {"path": "$tags", "includeArrayIndex": "tag_index"}},
    {"$unset": ["_id", "seller_id"]}
])
print_cursor(unwind_cursor)

{'name': 'Mug', 'tags': 'Home', 'tag_index': 0}

{'name': 'Mug', 'tags': 'Kitchen', 'tag_index': 1}

{'name': 'Pens', 'tags': 'Office', 'tag_index': 0}

{'name': 'Pens', 'tags': 'School', 'tag_index': 1}



### group

In [48]:
group_cursor = db.products.aggregate([
    {"$group": {"_id": "$tags"}}
])
print_cursor(group_cursor)

{'_id': ['Beauty']}

{'_id': ['Office', 'School']}

{'_id': ['Home', 'Kitchen']}



In [49]:
group_cursor = db.products.aggregate([
    {"$unwind": "$tags"},
    {"$group": {"_id": "$tags"}}
])
print_cursor(group_cursor)

{'_id': 'Beauty'}

{'_id': 'Kitchen'}

{'_id': 'Office'}

{'_id': 'Home'}

{'_id': 'School'}



### count (accumulator operator)

In [50]:
group_cursor = db.products.aggregate([
    {"$unwind": "$tags"},
    {"$group": {"_id": "$tags", "num_entries": {"$count": {}}}}
])
print_cursor(group_cursor)

{'_id': 'Beauty', 'num_entries': 4}

{'_id': 'Kitchen', 'num_entries': 1}

{'_id': 'Office', 'num_entries': 1}

{'_id': 'Home', 'num_entries': 1}

{'_id': 'School', 'num_entries': 1}



### sum (accumulator operator)

In [52]:
print_cursor(db.orders.find({}))

{'_id': ObjectId('642d06472c53af7ae4ee5b8d'), 'items': [{'product_id': ObjectId('642d06472c53af7ae4ee5b88'), 'quantity': 1}, {'product_id': ObjectId('642d06472c53af7ae4ee5b8a'), 'quantity': 1}]}

{'_id': ObjectId('642d06472c53af7ae4ee5b8e'), 'items': [{'product_id': ObjectId('642d06472c53af7ae4ee5b8b'), 'quantity': 1}, {'product_id': ObjectId('642d06472c53af7ae4ee5b8c'), 'quantity': 1}]}

{'_id': ObjectId('642d06472c53af7ae4ee5b8f'), 'items': [{'product_id': ObjectId('642d06472c53af7ae4ee5b89'), 'quantity': 5}, {'product_id': ObjectId('642d06472c53af7ae4ee5b87'), 'quantity': 1}]}

{'_id': ObjectId('642d06472c53af7ae4ee5b90'), 'items': [{'product_id': ObjectId('642d06472c53af7ae4ee5b88'), 'quantity': 2}, {'product_id': ObjectId('642d06472c53af7ae4ee5b8c'), 'quantity': 1}]}

{'_id': ObjectId('642d06472c53af7ae4ee5b91'), 'items': [{'product_id': ObjectId('642d06472c53af7ae4ee5b88'), 'quantity': 1}]}

{'_id': ObjectId('642d06472c53af7ae4ee5b92'), 'items': [{'product_id': ObjectId('642d0647

In [51]:
group_cursor = db.orders.aggregate([
    {"$unwind": "$items"},
    {"$group": {"_id": "$items.product_id", "total_quantity": {"$sum": "$items.quantity"}}}
])
print_cursor(group_cursor)

{'_id': ObjectId('642d06472c53af7ae4ee5b89'), 'total_quantity': 5}

{'_id': ObjectId('642d06472c53af7ae4ee5b8a'), 'total_quantity': 2}

{'_id': ObjectId('642d06472c53af7ae4ee5b8b'), 'total_quantity': 1}

{'_id': ObjectId('642d06472c53af7ae4ee5b87'), 'total_quantity': 1}

{'_id': ObjectId('642d06472c53af7ae4ee5b8c'), 'total_quantity': 2}

{'_id': ObjectId('642d06472c53af7ae4ee5b88'), 'total_quantity': 4}



### first, last (accumulator operators)

In [53]:
group_cursor = db.products.aggregate([
    {"$group": {"_id": "$tags", "num_entries": {"$count": {}}, "first": {"$first": "$name"}, "last": {"$last": "$name"}}}
])
print_cursor(group_cursor)

{'_id': ['Home', 'Kitchen'], 'num_entries': 1, 'first': 'Mug', 'last': 'Mug'}

{'_id': ['Office', 'School'], 'num_entries': 1, 'first': 'Pens', 'last': 'Pens'}

{'_id': ['Beauty'], 'num_entries': 4, 'first': 'Moisturizer', 'last': 'Eyeliner'}



### push (accumulator operator)

In [54]:
group_cursor = db.products.aggregate([
    {"$group": {"_id": "$tags", "products": {"$push": "$name"}}}
])
print_cursor(group_cursor)

{'_id': ['Beauty'], 'products': ['Moisturizer', 'Face Cleanser', 'Concealer Makeup', 'Eyeliner']}

{'_id': ['Home', 'Kitchen'], 'products': ['Mug']}

{'_id': ['Office', 'School'], 'products': ['Pens']}



### addToSet (accumulator operator)

In [55]:
group_cursor = db.products.aggregate([
    {"$group": {"_id": "$tags", "products": {"$addToSet": "$name"}}}
])
print_cursor(group_cursor)

{'_id': ['Beauty'], 'products': ['Eyeliner', 'Concealer Makeup', 'Moisturizer', 'Face Cleanser']}

{'_id': ['Home', 'Kitchen'], 'products': ['Mug']}

{'_id': ['Office', 'School'], 'products': ['Pens']}



### $$ROOT (system variable)
References the root document, i.e. the top-level document.

In [56]:
group_cursor = db.products.aggregate([
    {"$group": {"_id": "$tags", "products": {"$addToSet": "$$ROOT"}}}
])
print_cursor(group_cursor)

{'_id': ['Home', 'Kitchen'], 'products': [{'_id': ObjectId('642d06472c53af7ae4ee5b87'), 'name': 'Mug', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Home', 'Kitchen']}]}

{'_id': ['Office', 'School'], 'products': [{'_id': ObjectId('642d06472c53af7ae4ee5b89'), 'name': 'Pens', 'seller_id': ObjectId('642d06472c53af7ae4ee5b82'), 'tags': ['Office', 'School']}]}

{'_id': ['Beauty'], 'products': [{'_id': ObjectId('642d06472c53af7ae4ee5b8b'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('642d06472c53af7ae4ee5b84'), 'tags': ['Beauty']}, {'_id': ObjectId('642d06472c53af7ae4ee5b8c'), 'name': 'Eyeliner', 'seller_id': ObjectId('642d06472c53af7ae4ee5b85'), 'tags': ['Beauty']}, {'_id': ObjectId('642d06472c53af7ae4ee5b88'), 'name': 'Moisturizer', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Beauty']}, {'_id': ObjectId('642d06472c53af7ae4ee5b8a'), 'name': 'Face Cleanser', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Beauty']}]}



### addFields

In [58]:
add_fields_cursor = db.products.aggregate([
    {"$match": {"name": "Pens"}},
    {"$addFields": {"my_new_field": "hi there", "num_tags": {"$size": "$tags"}}}
])

print_cursor(add_fields_cursor)

{'_id': ObjectId('642d06472c53af7ae4ee5b89'), 'name': 'Pens', 'seller_id': ObjectId('642d06472c53af7ae4ee5b82'), 'tags': ['Office', 'School'], 'my_new_field': 'hi there', 'num_tags': 2}



### sample

In [59]:
sample_cursor = db.products.aggregate([
    {"$sample": {"size": 3}},
    {"$unset": ["_id", "seller_id"]}
])

print_cursor(sample_cursor)

{'name': 'Pens', 'tags': ['Office', 'School']}

{'name': 'Face Cleanser', 'tags': ['Beauty']}

{'name': 'Mug', 'tags': ['Home', 'Kitchen']}



### lookup

In [60]:
lookup_cursor = db.products.aggregate([
    {"$lookup": {
        "from": "users",
        "localField": "seller_id",
        "foreignField": "_id",
        "as": "sellers"
    }}
])

print_cursor(lookup_cursor)

{'_id': ObjectId('642d06472c53af7ae4ee5b87'), 'name': 'Mug', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Home', 'Kitchen'], 'sellers': [{'_id': ObjectId('642d06472c53af7ae4ee5b81'), 'name': 'Sarah'}]}

{'_id': ObjectId('642d06472c53af7ae4ee5b88'), 'name': 'Moisturizer', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Beauty'], 'sellers': [{'_id': ObjectId('642d06472c53af7ae4ee5b81'), 'name': 'Sarah'}]}

{'_id': ObjectId('642d06472c53af7ae4ee5b89'), 'name': 'Pens', 'seller_id': ObjectId('642d06472c53af7ae4ee5b82'), 'tags': ['Office', 'School'], 'sellers': [{'_id': ObjectId('642d06472c53af7ae4ee5b82'), 'name': 'Bob'}]}

{'_id': ObjectId('642d06472c53af7ae4ee5b8a'), 'name': 'Face Cleanser', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Beauty'], 'sellers': [{'_id': ObjectId('642d06472c53af7ae4ee5b81'), 'name': 'Sarah'}]}

{'_id': ObjectId('642d06472c53af7ae4ee5b8b'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('642d06472c53af7ae4ee5b84'),

In [61]:
lookup_cursor = db.products.aggregate([
    {"$lookup": {
        "from": "users",
        "localField": "seller_id",
        "foreignField": "_id",
        "as": "sellers"
    }},
    {"$project": {"_id": 0, "product_name": "$name", "seller_name": {"$first": "$sellers.name"}}}
])

print_cursor(lookup_cursor)

{'product_name': 'Mug', 'seller_name': 'Sarah'}

{'product_name': 'Moisturizer', 'seller_name': 'Sarah'}

{'product_name': 'Pens', 'seller_name': 'Bob'}

{'product_name': 'Face Cleanser', 'seller_name': 'Sarah'}

{'product_name': 'Concealer Makeup', 'seller_name': 'Lisa'}

{'product_name': 'Eyeliner', 'seller_name': 'Jessica'}



In [63]:
lookup_cursor = db.users.aggregate([
    {"$lookup": {
        "from": "products",
        "localField": "_id",
        "foreignField": "seller_id",
        "as": "products"
    }},
    {"$project": {"_id": 0, "seller_name": "$name", "products": "$products.name"}}
])

print_cursor(lookup_cursor)

{'seller_name': 'Sarah', 'products': ['Mug', 'Moisturizer', 'Face Cleanser']}

{'seller_name': 'Bob', 'products': ['Pens']}

{'seller_name': 'Jose', 'products': []}

{'seller_name': 'Lisa', 'products': ['Concealer Makeup']}

{'seller_name': 'Jessica', 'products': ['Eyeliner']}

{'seller_name': 'Tina', 'products': []}



In [64]:
lookup_cursor = db.users.aggregate([
    {"$lookup": {
        "from": "products",
        "localField": "_id",
        "foreignField": "seller_id",
        "as": "products"
    }},
    {"$addFields": {"num_products": {"$size": "$products"}}},
    {"$match": {"num_products": {"$gte": 1}}},
    {"$project": {"_id": 0, "seller_name": "$name", "products": "$products.name"}}
])

print_cursor(lookup_cursor)

{'seller_name': 'Sarah', 'products': ['Mug', 'Moisturizer', 'Face Cleanser']}

{'seller_name': 'Bob', 'products': ['Pens']}

{'seller_name': 'Lisa', 'products': ['Concealer Makeup']}

{'seller_name': 'Jessica', 'products': ['Eyeliner']}



In [68]:
lookup_cursor = db.products.aggregate([
    {"$group": {"_id": "$seller_id", "product_name": {"$push": "$name"}}},
    {"$lookup": {
        "from": "users",
        "localField": "_id",
        "foreignField": "_id",
        "as": "sellers"
    }},
    {"$project": {"_id": 0, "seller_name": {"$first": "$sellers.name"}, "products": "$product_name"}}
])

print_cursor(lookup_cursor)

{'seller_name': 'Lisa', 'products': ['Concealer Makeup']}

{'seller_name': 'Jessica', 'products': ['Eyeliner']}

{'seller_name': 'Sarah', 'products': ['Mug', 'Moisturizer', 'Face Cleanser']}

{'seller_name': 'Bob', 'products': ['Pens']}



### unionWith

In [69]:
union_cursor = db.products.aggregate([
    {"$unionWith": "users"}
])

print_cursor(union_cursor)

{'_id': ObjectId('642d06472c53af7ae4ee5b87'), 'name': 'Mug', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Home', 'Kitchen']}

{'_id': ObjectId('642d06472c53af7ae4ee5b88'), 'name': 'Moisturizer', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b89'), 'name': 'Pens', 'seller_id': ObjectId('642d06472c53af7ae4ee5b82'), 'tags': ['Office', 'School']}

{'_id': ObjectId('642d06472c53af7ae4ee5b8a'), 'name': 'Face Cleanser', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b8b'), 'name': 'Concealer Makeup', 'seller_id': ObjectId('642d06472c53af7ae4ee5b84'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b8c'), 'name': 'Eyeliner', 'seller_id': ObjectId('642d06472c53af7ae4ee5b85'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b81'), 'name': 'Sarah'}

{'_id': ObjectId('642d06472c53af7ae4ee5b82'), 'name': 'Bob'}

{'_id': ObjectId('64

### regexMatch (operator)

In [73]:
user_search = "IS"

regex_cursor = db.products.aggregate([
    {"$unionWith": "users"},
    {"$addFields": {"matched": {"$regexMatch": {"input": "$name", "regex": user_search, "options": "i"}}}},
    {"$match": {"matched": True}},
    {"$unset": ["matched"]}
])

print_cursor(regex_cursor)

{'_id': ObjectId('642d06472c53af7ae4ee5b88'), 'name': 'Moisturizer', 'seller_id': ObjectId('642d06472c53af7ae4ee5b81'), 'tags': ['Beauty']}

{'_id': ObjectId('642d06472c53af7ae4ee5b84'), 'name': 'Lisa'}



### out
*Note: You can potentially overwrite all your data in a collection with this stage, use with caution*

In [74]:
out_cursor = db.products.aggregate([
    {"$match": {"tags": "Beauty"}},
    {"$out": {"db": "aggregation_test", "coll": "beauty_products"}}
])

print_cursor(out_cursor)

### merge
*Note: You can potentially overwrite data within a collection with this stage, use with caution*

In [75]:
out_cursor = db.products.aggregate([
    {"$match": {"tags": "Beauty"}},
    {"$merge": {"into": {"db": "aggregation_test", "coll": "beauty_products"}}}
])

print_cursor(out_cursor)

In [77]:
out_cursor = db.products.aggregate([
    {"$match": {"tags": "Office"}},
    {"$project": {"test": "test"}},
    {"$merge": {
        "into": {"db": "aggregation_test", "coll": "beauty_products"},
        "on": "_id", # must specify a indexed attribute
        "whenMatched": "replace",
        "whenNotMatched": "discard" # default: insert, options: discard, fail
    }}
])

print_cursor(out_cursor)

### cond (operator)

In [4]:
cond_cursor = db.users.aggregate([
    {"$project": {
        "_id": 0,
        "name": 1,
        "is_bob": {
            "$cond": {
                "if": {"$eq": ["$name", "Bob"]},
                "then": True,
                "else": False
            }
        }
    }}
])

In [5]:
print_cursor(cond_cursor)

{'name': 'Sarah', 'is_bob': False}

{'name': 'Bob', 'is_bob': True}

{'name': 'Jose', 'is_bob': False}

{'name': 'Lisa', 'is_bob': False}

{'name': 'Jessica', 'is_bob': False}

{'name': 'Tina', 'is_bob': False}



In [6]:
cond_cursor = db.products.aggregate([
    {"$project": {
        "_id": 0,
        "name": 1,
        "is_beauty_product": {
            "$cond": {
                "if": {"$in": ["Beauty", "$tags"]},
                "then": True,
                "else": False
            }
        }
    }}
])

print_cursor(cond_cursor)

{'name': 'Mug', 'is_beauty_product': False}

{'name': 'Moisturizer', 'is_beauty_product': True}

{'name': 'Pens', 'is_beauty_product': False}

{'name': 'Face Cleanser', 'is_beauty_product': True}

{'name': 'Concealer Makeup', 'is_beauty_product': True}

{'name': 'Eyeliner', 'is_beauty_product': True}



### $$NOW (system variable)

In [7]:
cond_cursor = db.users.aggregate([
    {"$match": {"name": "Bob"}},
    {"$project": {"_id": 0, "name": 1, "date_joined": "$$NOW"}}
])

print_cursor(cond_cursor)

{'name': 'Bob', 'date_joined': datetime.datetime(2023, 4, 6, 22, 49, 45, 876000)}



### Date Operators

In [8]:
date_add_cursor = db.users.aggregate([
    {"$match": {"name": "Bob"}},
    {"$addFields": {"date_joined": "$$NOW"}},
    {"$project": {"_id": 0, "name": 1, "date_joined": 1, "premium_exp_date": {
        "$dateAdd": {"startDate": "$date_joined", "unit": "day", "amount": 7}
    }}}
])

print_cursor(date_add_cursor)

{'name': 'Bob', 'premium_exp_date': datetime.datetime(2023, 4, 13, 22, 52, 33, 399000)}



In [11]:
date_diff_cursor = db.users.aggregate([
    {"$match": {"name": "Bob"}},
    {"$addFields": {"date_joined": "$$NOW"}},
    {"$project": {"_id": 0, "name": 1, "date_joined": 1, "premium_exp_date": {
        "$dateAdd": {"startDate": "$date_joined", "unit": "week", "amount": 2}
    }}},
    {"$addFields": {"premium_days_left": {
        "$dateDiff": {"startDate": "$date_joined", "endDate": "$premium_exp_date", "unit": "day"}
    }}}
])

print_cursor(date_diff_cursor)

{'name': 'Bob', 'date_joined': datetime.datetime(2023, 4, 6, 22, 57, 57, 706000), 'premium_exp_date': datetime.datetime(2023, 4, 20, 22, 57, 57, 706000), 'premium_days_left': 14}



In [12]:
date_to_part_cursor = db.users.aggregate([
    {"$match": {"name": "Bob"}},
    {"$addFields": {"date_joined": "$$NOW"}},
    {"$project": {"_id": 0, "name": 1, "date_joined": 1, "date_joined_parts": {
        "$dateToParts": {"date": "$date_joined"}
    }}}
])

print_cursor(date_to_part_cursor)

{'name': 'Bob', 'date_joined': datetime.datetime(2023, 4, 6, 23, 0, 5, 98000), 'date_joined_parts': {'year': 2023, 'month': 4, 'day': 6, 'hour': 23, 'minute': 0, 'second': 5, 'millisecond': 98}}



In [13]:
date_year_cursor = db.users.aggregate([
    {"$match": {"name": "Bob"}},
    {"$addFields": {"date_joined": "$$NOW"}},
    {"$project": {"_id": 0, "name": 1, "date_joined": 1, "date_joined_year": {
        "$year": "$date_joined"
    }}}
])

print_cursor(date_year_cursor)

{'name': 'Bob', 'date_joined': datetime.datetime(2023, 4, 6, 23, 1, 25, 69000), 'date_joined_year': 2023}



### expr (operator)

In [18]:
### Without Expresion
exp_cursor = db.users.aggregate([
    {"$match": {"name": "Bob"}},
    {"$addFields": {"date_joined": "$$NOW"}},
    {"$addFields": {"premium_exp_date": {
        "$dateAdd": {"startDate": "$date_joined", "unit": "day", "amount": 7}
    }}},
    {"$addFields": {"premium_days_left": {
        "$dateDiff": {"startDate": "$date_joined", "endDate": "$premium_exp_date", "unit": "day"}
    }}},
    {"$match": {"premium_days_left": {"$lt": 10}}}
])

print_cursor(exp_cursor)

{'_id': ObjectId('642d06472c53af7ae4ee5b82'), 'name': 'Bob', 'date_joined': datetime.datetime(2023, 4, 6, 23, 22, 8, 12000), 'premium_exp_date': datetime.datetime(2023, 4, 13, 23, 22, 8, 12000), 'premium_days_left': 7}



In [19]:
### With Expresion
exp_cursor = db.users.aggregate([
    {"$match": {"name": "Bob"}},
    {"$addFields": {"date_joined": "$$NOW"}},
    {"$addFields": {"premium_exp_date": {
        "$dateAdd": {"startDate": "$date_joined", "unit": "day", "amount": 7}
    }}},
    {"$match": {"$expr": {
        "$lt": [{"$dateDiff": {"startDate": "$date_joined", "endDate": "$premium_exp_date", "unit": "day"}}, 10]
    }}}
])

print_cursor(exp_cursor)

{'_id': ObjectId('642d06472c53af7ae4ee5b82'), 'name': 'Bob', 'date_joined': datetime.datetime(2023, 4, 6, 23, 25, 22, 60000), 'premium_exp_date': datetime.datetime(2023, 4, 13, 23, 25, 22, 60000)}



### ifNull (operator)

In [25]:
if_null_cursor = db.users.aggregate([
    {"$project": {"_id": 0, "name": 1, "has_premium_exp_date": {
        "$ifNull": ["$premium_exp_date", "$name", False]  # Warks as COALESCE
    }}}
])

print_cursor(if_null_cursor)

{'name': 'Sarah', 'has_premium_exp_date': datetime.datetime(2022, 12, 1, 0, 0)}

{'name': 'Bob', 'has_premium_exp_date': datetime.datetime(2023, 4, 30, 0, 0)}

{'name': 'Jose', 'has_premium_exp_date': 'Jose'}

{'name': 'Lisa', 'has_premium_exp_date': 'Lisa'}

{'name': 'Jessica', 'has_premium_exp_date': 'Jessica'}

{'name': 'Tina', 'has_premium_exp_date': 'Tina'}

{'has_premium_exp_date': False}



### type (operator)

In [31]:
type_cursor = db.users.aggregate([
    {"$project": {
        "_id": 0, 
        "name": 1, 
        "name_type": {"$type": "$name"},
        "premium_exp_date_type": {"$type": "$premium_exp_date"},
        "_id_type": {"$type": "$_id"} 
    }}
])

print_cursor(type_cursor)

{'name': 'Sarah', 'name_type': 'string', 'premium_exp_date_type': 'date', '_id_type': 'objectId'}

{'name': 'Bob', 'name_type': 'string', 'premium_exp_date_type': 'date', '_id_type': 'objectId'}

{'name': 'Jose', 'name_type': 'string', 'premium_exp_date_type': 'missing', '_id_type': 'objectId'}

{'name': 'Lisa', 'name_type': 'string', 'premium_exp_date_type': 'missing', '_id_type': 'objectId'}

{'name': 'Jessica', 'name_type': 'string', 'premium_exp_date_type': 'missing', '_id_type': 'objectId'}

{'name': 'Tina', 'name_type': 'string', 'premium_exp_date_type': 'missing', '_id_type': 'objectId'}

{'name_type': 'missing', 'premium_exp_date_type': 'missing', '_id_type': 'objectId'}



### switch (operator)

In [34]:
switch_cursor = db.users.aggregate([
    {"$project": {
        "_id": 0, 
        "name": 1, 
        "user_account_error": {
            "$switch": {
                "branches": [
                    {"case": {"$eq": [{"$type": "$name"}, "missing"]}, "then": "Missing Name"},
                    {"case": {"$eq": [{"$type": "$premium_exp_date"}, "missing"]}, "then": "Missing Premium Expiration Date"},
                    {"case": {"$lt": ["$premium_exp_date", "$$NOW"]}, "then": "Premium Expiration Date Expired"}
                ],
                "default": "No Errors Found."
            }
        }
    }
}
])

print_cursor(switch_cursor)

{'name': 'Sarah', 'user_account_error': 'Premium Expiration Date Expired'}

{'name': 'Bob', 'user_account_error': 'No Errors Found.'}

{'name': 'Jose', 'user_account_error': 'Missing Premium Expiration Date'}

{'name': 'Lisa', 'user_account_error': 'Missing Premium Expiration Date'}

{'name': 'Jessica', 'user_account_error': 'Missing Premium Expiration Date'}

{'name': 'Tina', 'user_account_error': 'Missing Premium Expiration Date'}

{'user_account_error': 'Missing Name'}

